# Day 18: Lavaduct Lagoon

In [1]:
test_data_raw = """R 6 (#70c710)
D 5 (#0dc571)
L 2 (#5713f0)
D 2 (#d2c081)
R 2 (#59c680)
D 2 (#411b91)
L 5 (#8ceee2)
U 2 (#caa173)
L 1 (#1b58a2)
U 2 (#caa171)
R 2 (#7807d2)
U 3 (#a77fa3)
L 2 (#015232)
U 2 (#7a21e3)"""


def parse_data(data_raw):
    data = []
    for line in data_raw.splitlines():
        splitted = line.split(" ")
        direction = splitted[0]
        distance = int(splitted[1])
        data.append((direction, distance))
    return data


test_data = parse_data(test_data_raw)
test_data

[('R', 6),
 ('D', 5),
 ('L', 2),
 ('D', 2),
 ('R', 2),
 ('D', 2),
 ('L', 5),
 ('U', 2),
 ('L', 1),
 ('U', 2),
 ('R', 2),
 ('U', 3),
 ('L', 2),
 ('U', 2)]

In [18]:
import numpy as np


def print_dig(terrain):
    for row in terrain:
        print("".join(["#" if x else "." for x in row]))


def expand_terrain(terrain, position, direction, distance):
    axis = 0 if direction in "UD" else 1
    sign = 1 if direction in "RD" else -1
    new_position_along_axis = position[axis] + sign * distance
    new_size_along_axis = max(terrain.shape[axis], new_position_along_axis + 1) - min(0, new_position_along_axis)

    if new_size_along_axis > terrain.shape[axis]:
        base_ext_shape = new_size_along_axis - terrain.shape[axis], terrain.shape[1 - axis]
        ext_shape = base_ext_shape[axis], base_ext_shape[1 - axis]
        ext = np.zeros(ext_shape, dtype=bool)
        concat = (terrain, ext) if direction in "RD" else (ext, terrain)
        terrain = np.concatenate(concat, axis=axis)

        if sign == -1:
            new_position = list(position)
            new_position[axis] += ext_shape[axis]
            position = tuple(new_position)


    return terrain, position



for terrain, position, direction, distance in [
    (np.zeros((14, 1), dtype=bool), (2, 0), "R", 2),
    (np.zeros((14, 2), dtype=bool), (2, 2), "L", 5),
    (np.zeros((2, 14), dtype=bool), (0, 0), "U", 5),
]:
    new_terrain, new_position = expand_terrain(terrain, position, direction, distance)
    print("OLD SHAPE", terrain.shape)
    print("NEW SHAPE", new_terrain.shape)
    print(new_position)
    print()

OLD SHAPE (14, 1)
NEW SHAPE (14, 3)
(2, 0)

OLD SHAPE (14, 2)
NEW SHAPE (14, 5)
(2, 5)

OLD SHAPE (2, 14)
NEW SHAPE (7, 14)
(5, 0)



In [19]:
import numpy as np


def dig(terrain, position, direction, distance):
    DIRECTION_DICT = {"R": (0, 1), "L": (0, -1), "U": (-1, 0), "D": (1, 0)}
    
    terrain, position = expand_terrain(terrain, position, direction, distance)

    d = DIRECTION_DICT[direction]
    new_position = position[0] + distance * d[0], position[1] + distance * d[1]
    start_x = min(position[0], new_position[0])
    end_x = max(position[0], new_position[0])
    start_y = min(position[1], new_position[1])
    end_y = max(position[1], new_position[1])
    terrain[start_x:end_x + 1, start_y:end_y + 1] = True

    return terrain, new_position


def print_dig(terrain):
    for row in terrain:
        print("".join(["#" if x else "." for x in row]))


for terrain, position, direction, distance in [
    (np.zeros((4, 1), dtype=bool), (0, 0), "R", 2),
    (np.zeros((4, 1), dtype=bool), (0, 0), "L", 2),
    (np.zeros((2, 5), dtype=bool), (0, 0), "D", 4),
    (np.zeros((2, 5), dtype=bool), (0, 0), "U", 3),
]:
    new_terrain, new_position = dig(terrain, position, direction, distance)
    print_dig(new_terrain)
    print(new_position)
    print()

###
...
...
...
(0, 2)

###
...
...
...
(0, 0)

#....
#....
#....
#....
#....
(4, 0)

#....
#....
#....
#....
.....
(0, 0)



In [28]:
def dig_all(dig_data):
    position = (0, 0)
    terrain = np.zeros((1, 1), dtype=bool)
    for d in dig_data:
        direction, distance = d
        terrain, position = dig(terrain, position, direction, distance)
    return terrain, position


print_dig(dig_all(test_data)[0])

#######
#.....#
###...#
..#...#
..#...#
###.###
#...#..
##..###
.#....#
.######


In [43]:
def fill_dig(terrain, x=0, y=0):
    def visit(x, y, unvisited):
        if terrain[x, y]:
            return
        terrain[x, y] = True
        for x_dir, y_dir in [(1, 0), (-1, 0), (0, 1), (0, -1)]:
            xx, yy = x + x_dir, y + y_dir
            if 0 <= xx < terrain.shape[0] and 0 <= yy < terrain.shape[1]:
                unvisited.append((xx, yy))

    terrain = terrain.copy()
    unvisited = [(x + 1, y + 1)]
    while len(unvisited) > 0:
        x, y = unvisited.pop()
        visit(x, y, unvisited)

    return terrain


print_dig(fill_dig(dig_all(test_data)[0]))

#######
#######
#######
..#####
..#####
#######
#####..
#######
.######
.######


In [44]:
def part1(data):
    terrain, position = dig_all(data)
    terrain = fill_dig(terrain, *position)
    return int(np.sum(terrain))


part1(test_data)

62

In [25]:
with open("input.txt") as f:
    data = parse_data(f.read())

data[:5]

[('R', 2), ('U', 6), ('R', 4), ('U', 2), ('R', 6)]

In [45]:
part1(data)

42317